In [1]:
label_dict={0:"with_mask",1:"without_mask"}

In [2]:
categories=["with_mask","without_mask"]

In [3]:
labels=[0,1]

In [4]:
data_path="C:\\Users\\USER\\Downloads\\face-mask-detector-master\\face-mask-detector-master\\resources\\dataset"

In [5]:
import cv2,os

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.8 from "C:\Users\USER\anaconda3\python.exe"
  * The NumPy version is: "1.18.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


ImportError: numpy.core.multiarray failed to import

In [ ]:
data=[]
target=[]

In [ ]:
for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        try: 
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resized=cv2.resize(gray,(100,100))
            data.append(resized)
            target.append(label_dict[category])
        except Exception as e:
            pass

In [ ]:
import numpy as np

In [ ]:
data=np.array(data)

In [ ]:
data=data/255

In [ ]:
data.shape

In [ ]:
data=np.reshape(data,(data.shape[0],100,100,1))

In [ ]:
data.shape

In [ ]:
data.shape[1:]

In [ ]:
target=np.array(target)

In [ ]:
target.shape

In [ ]:
from keras.utils import np_utils

In [ ]:
new_target=np_utils.to_categorical(target)

In [ ]:
new_target.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D,MaxPooling2D

In [ ]:
model = Sequential()
model.add(Conv2D(200,(3,3),input_shape=data.shape[1:],activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(100, (3, 3), activation='relu')) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation="relu"))
model.add(Dense(2,activation='softmax'))

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
checkpoint=ModelCheckpoint("model-{epoch:03d}.model",save_best_only=True,mode="auto")
history=model.fit(train_data,train_target,epochs=20,validation_split=0.2,callbacks=[checkpoint])

In [ ]:
face_cascader=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [ ]:
cap=cv2.VideoCapture(0)
while True:
    _,img1=cap.read()
    gray1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    faces=face_cascader.detectMultiScale(gray1)
    pridiction=model.predict(faces)
    c=0
    for (x,y,w,h) in faces:
        cv2.rectangle(img1,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.rectangle(img1,(x,y-40),(x+w,y),(0,0,255),2)
        cv2.putText(img1,label_dict[pridiction[c]],(x+2,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,0),2)
        c=c+1
    cv2.imshow("img",img1)
    cv2.waitKey(30)
cap.release()